In [11]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import os
import pycountry  

In [12]:
path_csv = './data/source/experiencias_por_pais.csv'
df = pd.read_csv(path_csv)

In [13]:
df.head()

,ID de Experiencia,Experiencia,Categoría,Ubicación,País,Duración,Precio,Disponibilidad
0,1,Cena gourmet en un restaurante de lujo,Gastronomía,Buenos Aires,Argentina,2 horas,150,Todo el año
1,2,Escapada de fin de semana a la playa,Aventura,Viña del Mar,Argentina,2 días,300,Verano
2,3,Clase de cocina internacional,Culinaria,Córdoba,Argentina,3 horas,75,Todo el año
3,4,Trekking en la montaña,Aventura,Mendoza,Argentina,1 día,100,Primavera
4,5,Tour enológico por viñedos,Cultural,San Juan,Argentina,6 horas,120,Todo el año


In [14]:
# Función auxiliar para aplicar indentación al XML
def indent_xml(elem, level=0):
    i = "\n" + level * "  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for subelem in elem:
            indent_xml(subelem, level + 1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [19]:
# Mapeo de nombres de países en español a nombres en inglés
country_map = {
    'Argentina': 'Argentina',
    'Uruguay': 'Uruguay',
    'Chile': 'Chile',
    'Perú': 'Peru',
    'Bolivia': 'Bolivia',
    'México': 'Mexico' 
}

In [20]:
def generar_xml_desde_df(df, output_file, country):
    # Crear el elemento raíz 'rss' con los atributos requeridos
    root = ET.Element('rss')
    root.set('xmlns:g', 'http://base.google.com/ns/1.0')
    root.set('version', '2.0')
    root.set('xmlns:atom', 'http://www.w3.org/2005/Atom')

    # Crear el elemento 'channel' dentro del root
    channel = ET.SubElement(root, 'channel')

    # Agregar el título del canal
    title = ET.SubElement(channel, 'title')
    title.text = f'Smartbox - {country}'

    # Obtener el nombre en inglés usando el mapeo
    country_name_in_english = country_map.get(country)
    if not country_name_in_english:
        print(f"Error: País '{country}' no encontrado en el mapeo.")
        return

    # Obtener el código alfa-2 del país usando pycountry
    try:
        country_obj = pycountry.countries.get(name=country_name_in_english)
        if not country_obj:
            raise ValueError(f"País '{country_name_in_english}' no encontrado en pycountry.")
    except Exception as e:
        print(f"Error al obtener el código del país: {e}")
        return

    # Definir la URL condicional en función del país
    link = ET.SubElement(channel, 'link')
    if country in ['Uruguay', 'Perú', 'Argentina', 'México']:
        link.text = f'www.bigbox.com.{country_obj.alpha_2.lower()}'
    else:
        link.text = f'www.bigbox.{country_obj.alpha_2.lower()}'

    # Descripción del canal
    description = ET.SubElement(channel, 'description')
    description.text = f'Experiencias disponibles en {country}.'

    # Iterar sobre cada fila del DataFrame filtrado
    for _, row in df.iterrows():
        # Crear un nuevo elemento 'item' para cada experiencia
        item = ET.SubElement(channel, 'item')

        # Agregar subelementos correspondientes a las columnas del DataFrame
        ET.SubElement(item, 'id_experiencia').text = str(row['ID de Experiencia'])
        ET.SubElement(item, 'experiencia').text = row['Experiencia']
        ET.SubElement(item, 'categoria').text = row['Categoría']
        ET.SubElement(item, 'ubicacion').text = row['Ubicación']
        ET.SubElement(item, 'pais').text = row['País']
        ET.SubElement(item, 'duracion').text = row['Duración']
        ET.SubElement(item, 'precio').text = str(row['Precio'])
        ET.SubElement(item, 'disponibilidad').text = row['Disponibilidad']

    # Aplicar indentación para una mejor legibilidad
    indent_xml(root)

    # Crear el árbol XML con la raíz 'rss'
    tree = ET.ElementTree(root)

    # Escribir el archivo XML resultante con la estructura adecuada
    tree.write(output_file, encoding='utf-8', xml_declaration=True)

In [21]:
# Extraer la columna 'País' como una lista
paises  = df['País'].unique().tolist()
# Creo la carpeta final o target si es que no existe
path_target = './data/target/'
os.makedirs(os.path.dirname(path_target), exist_ok=True)
# Creo un archivo xmls por pais
for pais in paises :
    generar_xml_desde_df(df[df['País'] == pais], f'{path_target}{pais}.xml', pais)